In [ ]:
import sys
import os
from typing import List, Optional

sys.path.append(os.path.abspath(".."))

from sqlmodel import Field, SQLModel, Session
from sqlmodel import Relationship, create_engine


class Person(SQLModel, table=True):
    id: str = Field(primary_key=True)
    parent_id: Optional[str] = Field(default=None, foreign_key="person.id")
    name: str = Field()
    parent: Optional["Person"] = Relationship(
        back_populates="children", sa_relationship_kwargs={"remote_side": "Person.id"}
    )
    children: List["Person"] = Relationship(
        back_populates="parent",
        sa_relationship_kwargs={"remote_side": "Person.parent_id"},
    )


engine = create_engine(
    "sqlite:///:memory:", echo=False, connect_args={"check_same_thread": False}
)

SQLModel.metadata.create_all(engine)

In [2]:
with Session(engine) as session:
    p = Person(
        id="1",
        name="Dad",
    )

    session.add(p)
    session.commit()

In [3]:
with Session(engine) as session:

    dad = session.get(Person, "1")
    print(dad)

name='Dad' parent_id=None id='1'


In [4]:
with Session(engine) as session:
    c = Person(id="3", name="Mia", parent_id="1")
    session.add(c)
    session.commit()
    session.refresh(c)

In [5]:
with Session(engine) as session:
    m = session.get(Person, "3")
    assert m
    print("Mia")
    print(m)
    print(f"parent={m.parent}")
    print(f"children={m.children}")

    d = session.get(Person, "1")
    assert d
    print("Dad")
    print(d)
    print(f"parent={d.parent}")
    print(f"children={d.children}")

Mia
name='Mia' parent_id='1' id='3'
parent=name='Dad' parent_id=None id='1'
children=[]
Dad
name='Dad' parent_id=None id='1'
parent=None
children=[Person(name='Mia', parent_id='1', id='3')]
